In [2]:

# Jesus Ernesto Estrada Valenzuela iA 2025

import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

# 1. Cargar y preparar el conjunto de datos MNIST
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Convertir a float32 para evitar warnings y normalizar
x_train = tf.cast(x_train / 255.0, tf.float32)
x_test = tf.cast(x_test / 255.0, tf.float32)

# Agrega una dimension de canales
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

# 2. Utiliza tf.data para separar por lotes y mezclar
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

# 3. Construye el modelo utilizando la API de Keras model subclassing
class MyModel(Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.conv1 = Conv2D(32, 3, activation='relu')
    self.flatten = Flatten()
    self.d1 = Dense(128, activation='relu')
    self.d2 = Dense(10, activation='softmax')

  def call(self, x):
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)

# Crea una instancia del modelo
model = MyModel()

# 4. Escoge un optimizador y una funcion de perdida
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

# 5. Escoge metricas para medir la perdida y exactitud
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

# 6. Utiliza tf.GradientTape para entrenar el modelo
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    predictions = model(images)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

# 7. Define el paso de prueba
@tf.function
def test_step(images, labels):
  predictions = model(images)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

# 8. Ejecuta el bucle de entrenamiento
EPOCHS = 5

for epoch in range(EPOCHS):
  # Bucle de entrenamiento
  for images, labels in train_ds:
    train_step(images, labels)

  # Bucle de prueba (validación)
  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  template = 'Epoch {}, Perdida: {}, Exactitud: {}, Perdida de prueba: {}, Exactitud de prueba: {}'
  print(template.format(epoch+1,
                        train_loss.result(),
                        train_accuracy.result()*100,
                        test_loss.result(),
                        test_accuracy.result()*100))

  # --- LA CORRECCIÓN ESTÁ AQUÍ ---
  # Reinicia las metricas para el siguiente epoch.
  train_loss.reset_state()
  train_accuracy.reset_state()
  test_loss.reset_state()
  test_accuracy.reset_state()

Epoch 1, Perdida: 0.13677383959293365, Exactitud: 95.93000030517578, Perdida de prueba: 0.05964800342917442, Exactitud de prueba: 98.07999420166016
Epoch 2, Perdida: 0.04030090570449829, Exactitud: 98.75, Perdida de prueba: 0.05217477306723595, Exactitud de prueba: 98.19999694824219
Epoch 3, Perdida: 0.02035067416727543, Exactitud: 99.36166381835938, Perdida de prueba: 0.06035580486059189, Exactitud de prueba: 98.2699966430664
Epoch 4, Perdida: 0.012733658775687218, Exactitud: 99.5616683959961, Perdida de prueba: 0.05308060348033905, Exactitud de prueba: 98.43000030517578
Epoch 5, Perdida: 0.009102367796003819, Exactitud: 99.68499755859375, Perdida de prueba: 0.06761150807142258, Exactitud de prueba: 98.31999969482422
